In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Epoch 공모전/2020_2023_최종데이터.csv")
df.drop(['tm_dt'], axis=1, inplace=True)

In [ ]:
# 1. X, y 분리
X = df.drop("call_count", axis=1)
y = df["call_count"]

In [ ]:
# 2. 인코딩 + 스케일링
X = pd.get_dummies(X)
y = pd.to_numeric(y, errors='coerce')
print(y.dtype)

int64


In [ ]:
# 데이터 분리 및 스케일링
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 모델 정의
model = Sequential([
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(1)
])

# 컴파일
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# 조기 종료
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 학습
history = model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


859/859 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 4.5082 - mae: 1.0165 - val_loss: 2.4890 - val_mae: 0.8591
Epoch 2/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.7915 - mae: 0.8972 - val_loss: 2.3311 - val_mae: 0.8258
Epoch 3/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3.1825 - mae: 0.8855 - val_loss: 2.1000 - val_mae: 0.8675
Epoch 4/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.6968 - mae: 0.8736 - val_loss: 2.0365 - val_mae: 0.8071
Epoch 5/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2.4264 - mae: 0.8578 - val_loss: 1.9216 - val_mae: 0.8259
Epoch 6/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.8787 - mae: 0.8753 - val_loss: 2.0135 - val_mae: 0.8355
Epoch 7/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 2.2003 - mae: 0.8501 - val_loss: 1.9578 - val_mae: 0.8026
Epoch 8/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 2.6597 - mae: 0.8751 - val_loss: 2.0036 - val_mae: 0.8345
Epoch 9/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss:

In [ ]:
# 검증 데이터 로드
valid_df = pd.read_csv("/content/drive/MyDrive/Epoch 공모전/2024_최종데이터.csv")

# 훈련 데이터에는 없던 열 제거
valid_df.drop(['tm_dt'], axis=1, inplace=True)

# X 분리
X_valid = valid_df.drop(columns=['call_count'])  # 예측 대상 제외

# 인코딩: 훈련과 동일하게 One-Hot
X_valid = pd.get_dummies(X_valid)

# 누락된 열 보완 및 순서 맞추기
X_valid = X_valid.reindex(columns=X_train.columns, fill_value=0)

# 정규화 (훈련 스케일러 재사용)
X_valid_scaled = scaler.transform(X_valid)

# 예측
y_valid_pred = model.predict(X_valid_scaled)
y_valid_pred = np.round(y_valid_pred).astype(int).flatten()

# 예측값 덮어쓰기
valid_df['call_count'] = y_valid_pred

# 상위 결과 확인
from IPython.display import display
display(valid_df[['call_count']].head(10))

301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,call_count
0,1
1,2
2,2
3,1
4,1
5,2
6,2
7,1
8,2
9,2


In [ ]:
result_df = pd.read_csv('/content/drive/MyDrive/Epoch 공모전/test_call119.csv', encoding='euc-kr')

In [ ]:
result_df['call_count'] = valid_df['call_count']

In [ ]:
result_df.head()

,TM,address_city,address_gu,sub_address,STN,ta_max,ta_min,ta_max_min,hm_min,hm_max,ws_max,ws_ins_max,rn_day,call_count
0,20240501,부산광역시,강서구,대저2동,904,18.5,11.1,7.4,42.5,82.5,6.5,11.6,0.0,1
1,20240501,부산광역시,강서구,생곡동,904,18.5,11.1,7.4,42.5,82.5,6.5,11.6,0.0,2
2,20240501,부산광역시,강서구,송정동,937,16.9,9.9,7.0,55.3,93.9,4.5,9.7,0.0,2
3,20240501,부산광역시,강서구,신호동,950,16.6,11.4,5.2,48.1,84.6,6.4,13.5,0.0,1
4,20240501,부산광역시,금정구,구서동,940,16.9,10.2,6.7,46.8,91.3,3.3,8.7,0.0,1


In [ ]:
result_df.to_csv('/content/drive/MyDrive/Epoch 공모전/250028.csv', index=False, encoding='euc-kr')